We begin by importing all the necssary packages and modules. Next our task is to build the Machine_Replacement environment. For doing that, we create a class named as Machine_Replacement which accepts the number_of_states(nS), number_of_actions(nA) and replacement_cost(rep_cost) as input and generate the environment. Later we just need to cal the function gen_probability() and gen_expected_reward_function() to get the Probability distribution matrix and Reward matrix

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import numpy as np
import pandas as pd
import pickle
from itertools import product
import multiprocessing as mp
mp.set_start_method('spawn',True)
torch.multiprocessing.set_start_method('spawn',True)
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Machine_Replacement:
    def __init__(self,rep_cost=0.7,nS=6,nA=2):
        self.nS = nS;
        self.nA = nA;
        self.cost = np.linspace(0.1, 0.99,nS);
        self.rep_cost = rep_cost;
    def gen_probability(self):
        self.P = np.zeros((self.nA,self.nS,self.nS));
        for i in range(self.nS):
            for j in range(self.nS):
                if(i<=j):
                    self.P[0,i,j]=(i+1)*(j+1);
                else:
                    continue;
            self.P[0,i,:]=self.P[0,i,:]/np.sum(self.P[0,i,:])
            self.P[1,i,0]=1;
        return self.P;
    def gen_reward(self):
        self.R=np.zeros((self.nA,self.nS,self.nS));
        for i in range(self.nS):
            self.R[0,i,:] = self.cost[i];
            self.R[1,i,0] = self.rep_cost+self.cost[0];
        return self.R;
    def gen_expected_reward(self):
        self.R = np.zeros((self.nA,self.nS));
        for i in range(self.nS):
            self.R[0,i] = self.cost[i];
            self.R[1,i] = self.rep_cost + self.cost[0];
        return self.R;

Next we create another class specifying the hyperparameters that might be required in our algorithm. Later when required we can just call ret_hyperparameters() to get the hyperparameters.

In [2]:
class get_hyperparameters:
    def __init__(self):
        self.T = 50000;
        self.runs = 5;
        self.lr = 0.1;
        self.batch_size = 50;
        self.start = 0;
        self.nS = 10;
        self.nA = 2;
        self.rep_cost = 0.7
        self.alpha = 0.5
        self.gamma = 0.95
    
    def ret_hyperparameters(self):
        return (self.T,self.runs,self.lr,self.batch_size,self.start,self.nS,self.nA,self.rep_cost,self.alpha,self.gamma)

Let us now define the pytorch model. So for doing that, we create a class named weights. There are 3 parameters, input_size which defines the number of perceptrons in the input layer. input_size = number of states(nS). The output size is the number of perceptrons in the output_layer. output_size = 1(which gives us the state distribution ratio for a particular state).

Later when we want to update or find the state distribution ratio of any state, just pass that state to the forward(). First that particular state is converted into one_hot vector and then fed to the network. Finally the network returns the output value as the ratio of state distribution.

In [3]:
class weights(nn.Module):
    def __init__(self,input_size,output_size,hidden_size = 0):
        super(weights,self).__init__()
        self.input_size = input_size;
        self.hidden_size = hidden_size;
        self.output_size = output_size;
        if(hidden_size!=0):
            self.linear1 = nn.Linear(self.input_size, self.hidden_size, bias=False)
            self.linear2 = nn.Linear(self.hidden_size, self.output_size, bias=False)
        else:
            self.linear1 = nn.Linear(self.input_size, self.output_size, bias=False)
    '''
        forward(): We accept a state 's' as input. Then we convert this into one hot encoding which is accomplished by first two lines.
        Further we convert this one_hot vector 's' into pytorch tensor and then pass it through the network to obtain a output which is returned 
    '''
    def forward(self,state):
        s = np.zeros(self.input_size);
        #print(state,end='===>');
        s[state] = 1;
        state = torch.FloatTensor(s).to(device)
        #print(state);
        if(self.hidden_size == 0):
            output = torch.exp(self.linear1(state)) #To ensure that the outputs are always positive. giving Relu will cause problems.
        else:
            output = torch.exp(self.linear2(torch.exp(self.linear1(state))));
        return output
    
    def forward_batch(self,states):
        s = np.zeros((len(states),self.input_size));
        #print(state,end='===>');
        for i in range(len(states)):
            s[i][states[i]] = 1
        states = torch.FloatTensor(s).to(device)
        #print(state);
        if(self.hidden_size == 0):
            output = torch.exp(self.linear1(states)) #To ensure that the outputs are always positive. giving Relu will cause problems.
        else:
            output = torch.exp(self.linear2(torch.exp(self.linear1(state))));
        return output
    
    def fast_forward(self,s1,ns1,s2,ns2):
        return self.forward(s1),self.forward(ns1),self.forward(s2),self.forward(ns2);

Now it is evident that the network written above will give me the ratio of state distribution. But the requirement is state distribution of the target_policy. So for doing that we need to to follow the below equation

state_distribution_of_target_policy = Normalize(state_distribution_ratio_obtained_from_network * behaviour_policy_state_distribution).

Now to get the target_policy_state_distribution, we need to obtain the behaviour_policy_state_distribution. To find that we use the class below. Now using the 

In [4]:
class beh_pol_sd:
    def __init__(self,P,policy,nS,nA):
        self.P = P
        self.policy = policy
        self.nS = nS;
        self.nA = nA;
    
    def onehot(self):
        pol = np.zeros((self.nS,self.nA));
        for i in range(self.nS):
            pol[i][int(self.policy[i])]=1;
        return pol;
    def find_transition_matrix(self,onehot_encode=1):
        if(onehot_encode==1):
            self.policy = self.onehot()
        T_s_s_next = np.zeros((self.nS,self.nS));
        for s in range(self.nS):
            for s_next in range(self.nS):
                for a in range(self.nA):
                    #print(s,s_next,a);
                    #print(T[a,s,s_next]);
                    T_s_s_next[s,s_next]+=self.P[a,s,s_next]*self.policy[s,a];
        return T_s_s_next;
    def state_distribution_simulated(self,onehot_encode=1):
        P_policy = self.find_transition_matrix(onehot_encode)
        #print(P_policy);
        P_dash = np.append(P_policy - np.eye(self.nS),np.ones((self.nS,1)),axis=1);
        #print(P_dash);
        P_last = np.linalg.pinv(np.transpose(P_dash))[:,-1]
        return P_last;

In [5]:
def one_hot(target_policy,nS,nA):
    one_hot_tp = [];
    for i in range(len(target_policy)):
        policy = target_policy[i];
        print(policy);
        tp=np.zeros((nS,nA));
        for j in range(nS):
            tp[j][policy[j]] = 1;
        one_hot_tp.append(tp);
    return np.array(one_hot_tp);

Now that we are all ready got our ingedients, let us define a separate class where we will define the average_case technique to find the state_distribution_ratio. Any object of the average_case class must possess the environment details such as number of states(nS) and number of actions(nA). The behaviour_policy, learning_rate,list_of_beta_values, a weight_object to refer to the weights class. The batch_size and the optimizer to be used(Adam_optimzer). 

Since in this variant, we will be having only 1 Neural Network and update the network for the sample that is obtained from the theta values. For this reason, we need to store all the beta values. Hence, we define our first function
1) find_beta(): This function finds all the beta(importance sampling) values. So beta values = target_policy[s,a]/behaviour_policy[s,a]. Now there are nPOL policies so for each policies we need to find the beta values for the ith target_policy, beta value will be beta[i,s,a]=target_policy[i,s,a]/behaviour_policy[s,a]. Now it is certain that a(action) can be {0,1} only. So instead of creating a new loop, we manually define two lines once for a=0 when beta value becomes beta[i,s,0]=target_policy[i,s,0]/behaviour_policy[s,0] and beta[i,s,1] = target_policy[i,s,1]/behaviour_policy[s,1]. Finally return all the beta values.

2) set_batch(): This function is used to set a data batch which is sampled from the behaviour_policy. Now the batch is set to be used for updating the state_distribution_ratio.

3) get_batch(): This function is used to get a random batch of 50 samples from the set data to be used to update our state_distribution_ratio. We actually create a batch of 50 data samples 10000 times in order to reach to a good value of state_distribution_ratio(Like in Linear regression we use a batch for several times until our gradient converges)

4) get_w(): This function is used to find the numerator and denominator of the loss function as mentioned in the paper 'Breaking the curse of horizon'. Now for finding the numerator paarameter pair = 0. To find the denominator pair = 1. Now it is observed that the denominator value easily goes to 0. So, to avoid divide by zero error, we add a small noise value of 0.000000001. This makes sure that the denominator value never goes to zero.

5) get_state_distribution_ratio(): This function uses the set data in the self.set_data() to get batches of size 50. Then calculate the loss using the equation mentioned in the paper 'Breaking the curse of horizon'. We use this calculated loss to update our weights of the Neural network by using Adam optimizer.

In [6]:
import numpy as np
class average_case_version_2:
  def __init__(self,nS,nA,behaviour_policy,state,lr,target_policy,batch_size,data_used):
    self.nS = nS;
    self.nPOL = nS
    self.nA = nA;
    self.behaviour_policy = behaviour_policy;
    self.lr = lr;
    self.beta = self.find_beta(target_policy,behaviour_policy)
    self.W_loss = 0
    self.weight_obj = weights(nS,1).to(device);
    self.batch_size = batch_size
    self.optimizerW = optim.Adam(self.weight_obj.parameters(),lr = self.lr);
    self.loss = [];
    self.Z = [];
    self.data_used = data_used
  def find_beta(self,target_policy,behaviour_policy):
    beta = np.zeros((self.nPOL,self.nS,self.nA))
    for i in range(self.nPOL):
      for s in range(self.nS):
        beta[i][s][0] = target_policy[i,s,0]/behaviour_policy[s,0];
        beta[i,s,1] = target_policy[i,s,1]/behaviour_policy[s,1];
    return beta;
  def set_batch(self,data):
        self.data = data;
        self.T = len(data);
  def get_batch(self):
      if(self.T<=50):
          return self.data
      else:
          i = 1;
          j = np.random.randint(0,self.T,self.batch_size);
          batch = [[self.data[k][0],self.data[k][1],self.data[k][2]] for k in j]
          '''while(i<=self.batch_size):
                if(np.random.random()<=0.5):
                    batch.append([self.data[j][0],self.data[j][1],self.data[j][2]])
                    j = (j+1)%self.T;
                i+=1;'''
          return batch; 
  def get_w(self,data,weight_obj,m,pair=0):
        if(pair == 1):
            Z_w_state = 0;
            for i in range(len(data)):
                val = weight_obj(data[i][0]);
                #print(val);
                Z_w_state+=val;
            #print(Z_w_state.detach().numpy()[0]/self.batch_size);
            Z_w_state = Z_w_state.cpu().detach().numpy()[0]/self.batch_size;
            if(Z_w_state<0.00000000000005):
                Z_w_state+=0.000000000001;
            return Z_w_state;
        else:
            state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2 = list(),list(),list(),list(),list(),list(),list(),list();
            K = list();
            for i in range(len(data)):
#                 print(data[i][0],data[i][1])
#                 print(np.shape(data))
                sample1 = data[i][0];
                sample2 = data[i][1];
                state1.append(sample1[0]);
                #print(sample1);
                w1,wn1,w2,wn2 = weight_obj.fast_forward(sample1[0],sample1[2],sample2[0],sample2[2]);
                w_state1.append(w1);
                w_next_state1.append(wn1);
                state2.append(sample2[0]);
                w_state2.append(w2);
                w_next_state2.append(wn2);
                #beta1.append(self.target_policy[sample1[0],sample1[1]]/self.behaviour_policy[sample1[0],sample1[1]]);
                beta1.append(self.beta[self.selected_policy,sample1[0],sample1[1]]);
                #beta2.append(self.target_policy[sample2[0],sample2[1]]/self.behaviour_policy[sample2[0],sample2[1]]);
                beta2.append(self.beta[self.selected_policy,sample2[0],sample2[1]])
                K.append(sample1[2]==sample2[2]);
            return (state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2,K);
        
  def get_w_updated(self,data,weight_obj,m,Z_w_state,pair=0): #data should be batch. note all inputs to be passed 
        if(pair == 1):
            Z_w_state = 0;
            for i in range(len(data)):
                val = weight_obj(data[i][0]);
                #print(val);
                Z_w_state+=val;
            #print(Z_w_state.detach().numpy()[0]/self.batch_size);
            Z_w_state = Z_w_state.cpu().detach().numpy()[0]/self.batch_size;
            if(Z_w_state<0.00000000000005):
                Z_w_state+=0.000000000001;
            return Z_w_state;
        else:
            data = torch.tensor(data)
#             print(data[:,0])
#             print(data[:,2])
#             print(data[:,0].size())
            new_data_current = torch.reshape(data[:,0],(50,1))
#             w_current_state = weight_obj.forward_batch(new_data_current)
            
            new_data_next = torch.reshape(data[:,2],(50,1))
            new_data = torch.cat((new_data_current,new_data_next),0)
            w = weight_obj.forward_batch(new_data)
            w = w/Z_w_state
#             print(w)
#             print(new_data.size())
#             print(w.size())
            w = torch.reshape(w,(2,50))
            beta = torch.zeros(50,1)
            for i in range(len(data)):
                beta[i] = self.beta[self.selected_policy,data[i][0],data[i][1]]
            
            K = torch.zeros(50,50)
            for i in range(len(data)):
                for j in range(len(data)):
                    K[i][j] = (data[i][2] == data[j][2])
#             print("here")
#             print(w.size())
#             print(beta.size())
#             print(K.size())
            w_0 = torch.reshape(w[0], (50,1))
            w_1 = torch.reshape(w[1], (50,1))
#             print(w_0)
#             print(beta)
#             print(w_1)
#             print(w_0 * beta - w_1)
#check here only
            delta = w_0 * beta - w_1
            temp1 = torch.matmul(K,delta) #50,50:50,1 = 50, 1
            final_sum = torch.matmul(torch.transpose(delta,0,1),temp1)
#             print(w_0)
#             print(w_1)
#             print(Z_w_state)
#             print(final_sum)
#             print(".........")
            return final_sum
            
            
            
            
#             state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2 = list(),list(),list(),list(),list(),list(),list(),list();
#             K = list();
#             for i in range(len(data)):
#                 sample1 = data[i][0];
#                 sample2 = data[i][1];
#                 state1.append(sample1[0]);
#                 #print(sample1);
#                 w1,wn1,w2,wn2 = weight_obj.fast_forward(sample1[0],sample1[2],sample2[0],sample2[2]);
#                 w_state1.append(w1);
#                 w_next_state1.append(wn1);
#                 state2.append(sample2[0]);
#                 w_state2.append(w2);
#                 w_next_state2.append(wn2);
#                 #beta1.append(self.target_policy[sample1[0],sample1[1]]/self.behaviour_policy[sample1[0],sample1[1]]);
#                 beta1.append(self.beta[self.selected_policy,sample1[0],sample1[1]]);
#                 #beta2.append(self.target_policy[sample2[0],sample2[1]]/self.behaviour_policy[sample2[0],sample2[1]]);
#                 beta2.append(self.beta[self.selected_policy,sample2[0],sample2[1]])
#                 K.append(sample1[2]==sample2[2]);
#             return (state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2,K);

    
  def get_state_distribution_ratio(self,selected_policy,run,t):
        batch = self.get_batch();
        eps = 0.04;
        self.data_used[run] =self.data_used[run]+batch;
        self.selected_policy = selected_policy
        pairs = list(product(batch,repeat=2));
        self.loss_episode = [];
#         print(np.shape(batch))
        for _ in range(5):
#             batch = self.get_batch();
            start = time.time()
            Z_w_state = self.get_w(batch, self.weight_obj, len(batch),1);
#             state1,state2,w_state1,w_state2,w_next_state1,w_next_state2,beta1,beta2,K = self.get_w(pairs, self.weight_obj, len(batch));
            self.w_loss = 0
            self.w_loss = self.get_w_updated(batch,self.weight_obj,len(batch),Z_w_state)
#             print(time.time() - start)
#             print("*******")
#             for i in range(len(state1)):
#                 self.w_loss+=(beta1[i]*(w_state1[i]/Z_w_state) - (w_next_state1[i]/Z_w_state))*(beta2[i]*(w_state2[i]/Z_w_state)-(w_next_state2[i]/Z_w_state))*K[i];
#             print("sourav")
#             print(self.w_loss)
            self.w_loss/=(2*self.batch_size);
            start = time.time()
            self.optimizerW.zero_grad();
            self.w_loss.backward(); #Improving the forward pass computation
            self.optimizerW.step();
            self.optimizerW.zero_grad();
#             print(time.time() - start)
            self.Z.append(Z_w_state)
            
        self.loss.append(self.w_loss.cpu().detach().numpy()[0]);
        state_dist=[];
        for i in range(self.nS):
            w_state = self.weight_obj(i);
            w_state = w_state.cpu().detach().numpy()[0];
            state_dist.append(w_state);
        return np.array(state_dist);

Instead of sampling state, action, next_state values on the go, we do it before hand and store it in a list named 'data'. So, that when required we can simply pass this data and save some time by before hand sampling the data.

In [7]:
def simulate_episode(T,state,behaviour_policy,P,batch_size):
  #global P,behaviour_policy,batch_size;
  data={};temp=[];
  for t in range(1,T+1):
    action = np.argmax(np.random.multinomial(1,behaviour_policy[state,:]))
    next_state = np.argmax(np.random.multinomial(1,P[action,state,:]));
    state = next_state;
    temp.append([state,action,next_state]);
    if(t%batch_size==0):
      data[int(t/batch_size)-1]=temp[:];
  return data;

For our help we create a softmax function.

In [8]:
def softmax(theta):
  theta = np.exp(theta);
  sum = np.sum(theta)
  return theta/sum;

We create a function naemd as preprocessing() which is responsible to perform the theta update. After every val=50 times, our learning rate is divided by 10. So if we start with lr = 1. After 50 instances, lr = 0.1. After 100 steps lr = 0.01 and so on...
Now at each instant, we found the softmax of the theta values. Now from the given probability values we sample a policy that will be updated next. 

In [9]:
import time
def processing(nPOL,run,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val,val2,behaviour_policy_state_distribution,beta,batch_size,alpha,data):
    lr = 1
    nS = nPOL;
    theta = np.ones(nPOL);
    c=0;
    rew = np.zeros(nPOL);
    S = np.ones(nPOL);
    F = np.ones(nPOL);
    n = np.ones(nPOL);
    for t in tqdm(range(1,T_update+1)):
      sampled_policy = np.argmin([np.random.beta(S[j],F[j]) for j in range(nPOL)])
      #policy_selected[t,run]=selected_policy;
      policy_sampled[t-1,run] = sampled_policy;
      #loss[t,run] = w_obj.w_loss.cpu().detach().numpy()[0];

      start = time.time()
      w_obj.set_batch(data[t-1]);
      sd = w_obj.get_state_distribution_ratio(sampled_policy,run,t-1);
      print(time.time() - start)
      sd = sd * behaviour_policy_state_distribution;
      sd = sd/np.sum(sd)
      rho_i = sum([sd[s]*R[target_policy[sampled_policy,s],s] for s in range(nS)]);
      rew[sampled_policy] = (1-alpha)*rew[sampled_policy] + alpha*rho_i
      S[sampled_policy]+=rew[sampled_policy];
      F[sampled_policy]=F[sampled_policy]+t-rew[sampled_policy]
      estimated_value[t-1,run] = rho_i;
    #print(data)

Finally call the hyperparameters, create the environment and call the pre-processing function.

In [11]:
if __name__ =='__main__':
    T,runs,lr,batch_size,state,nS,nA,rep_cost,alpha,gamma = get_hyperparameters().ret_hyperparameters();
    print(nS,nA);
    nPOL = nS;
    T_update = int(T/batch_size);
    mr_obj = Machine_Replacement(rep_cost,nS,nA);
    P,R = mr_obj.gen_probability(),mr_obj.gen_expected_reward()
    theta = np.ones(nPOL);
    behaviour_policy = np.ones((nS,nA))*0.5;
    print(behaviour_policy,P);
    behaviour_policy_state_distribution = beh_pol_sd(P,behaviour_policy,nS,nA).state_distribution_simulated(0);
    data = simulate_episode(T,state,behaviour_policy,P,batch_size);
    with open('data_used','wb') as f:
      pickle.dump(data,f);
    target_policy = np.ones((nPOL,nS),dtype = np.int8)
    data_dict={0:[],1:[],2:[],3:[],4:[]};
    val2 = []
    for i in range(nPOL-1,0,-1):
        target_policy[nPOL-i-1][0:i] = 0;
    for t in target_policy:
        val2.append(sum([R[t[s],s] for s in range(nS)]))
    val2 = np.array(val2);
    one_hot_target_policy = one_hot(target_policy,nS,nA)
    w_obj = average_case_version_2(nS,nA,behaviour_policy,state,lr,one_hot_target_policy,batch_size,data_dict);
    '''for i in range(nPOL):
      w_obj[i].set_target_policy(one_hot_target_policy[i]);'''
    policy_selected = np.zeros((T_update,runs))
    policy_sampled = np.zeros((T_update,runs))
    theta_change = []
    estimated_value = np.zeros((T_update,runs))
    loss = np.zeros((T_update,runs))
    val = 50;
    lr = 1;
    beta = 1;
    #######################################################################################
    '''mp.set_start_method('spawn')
    p1 = mp.Process(target = processing ,args=(nPOL,0,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val))
    p2 = mp.Process(target = processing ,args=(nPOL,1,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val))
    p3 = mp.Process(target = processing ,args=(nPOL,2,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val))
    p4 = mp.Process(target = processing ,args=(nPOL,3,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val))
    p5 = mp.Process(target = processing ,args=(nPOL,4,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val))
    #p1 = mp.Process(target = processing ,args=(0,policy_sampled,policy_selected,loss,estimated_value,T_update,val))
    #p1 = mp.Process(target = processing ,args=(0,policy_sampled,policy_selected,loss,estimated_value,T_update,val))
    #p1 = mp.Process(target = processing ,args=(0,policy_sampled,policy_selected,loss,estimated_value,T_update,val))
    #p1 = mp.Process(target = processing ,args=(0,policy_sampled,policy_selected,loss,estimated_value,T_update,val))
    #p1 = mp.Process(target = processing ,args=(0,policy_sampled,policy_selected,loss,estimated_value,T_update,val))
    p1.start();
    p2.start();
    p3.start();
    p4.start();
    p5.start();
    p1.join();
    p2.join();
    p3.join();
    p4.join();
    p5.join();'''
    #######################################################################################
    for run in tqdm(range(runs)):
        processing(nPOL,run,policy_sampled,policy_selected,loss,estimated_value,T_update,P,R,val,val2,behaviour_policy_state_distribution,beta,batch_size,alpha,data)
        print("One run completed");
    #######################################################################################
    #pd.DataFrame(policy_selected).to_excel("Policy_selection.xlsx");
    #pd.DataFrame(np.array(theta_change)).to_excel("Theta_values_non_estimate.xlsx");
    pd.DataFrame(policy_sampled).to_excel("Policy_sampling_Thompson_Sampling_variant_2_faster_10_states.xlsx");
    pd.DataFrame(data[T_update-1]).to_excel("Data_Used_Thompson_Sampling_variant_2_faster_10_states.xlsx");
    pd.DataFrame(estimated_value).to_excel("Estimated_Value_functions_Thompson_Sampling_variant_2_faster_10_states.xlsx");

10 2
[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]] [[[0.01818182 0.03636364 0.05454545 0.07272727 0.09090909 0.10909091
   0.12727273 0.14545455 0.16363636 0.18181818]
  [0.         0.03703704 0.05555556 0.07407407 0.09259259 0.11111111
   0.12962963 0.14814815 0.16666667 0.18518519]
  [0.         0.         0.05769231 0.07692308 0.09615385 0.11538462
   0.13461538 0.15384615 0.17307692 0.19230769]
  [0.         0.         0.         0.08163265 0.10204082 0.12244898
   0.14285714 0.16326531 0.18367347 0.20408163]
  [0.         0.         0.         0.         0.11111111 0.13333333
   0.15555556 0.17777778 0.2        0.22222222]
  [0.         0.         0.         0.         0.         0.15
   0.175      0.2        0.225      0.25      ]
  [0.         0.         0.         0.         0.         0.
   0.20588235 0.23529412 0.26470588 0.29411765]
  [0.         0.         0.         0.         0.         0.
   0.         0.29

  0%|                                          | 2/1000 [00:00<01:29, 11.15it/s]

0.09106826782226562
0.08799600601196289


0.08850359916687012



  0%|▏                                         | 4/1000 [00:00<01:29, 11.13it/s]

...

0.08667731285095215
0.09139561653137207
0.0865182876586914



100%|███████████████████████████████████████▊| 996/1000 [01:29<00:00, 11.24it/s]

0.09032988548278809
0.08449411392211914
0.09095048904418945



100%|█████████████████████████████████████████████| 5/5 [07:20<00:00, 88.14s/it]

0.08714842796325684
0.09092545509338379
0.08679604530334473
One run completed


In [23]:
!pip3 install openpyxl

You should consider upgrading via the '/home/user/anaconda3/bin/python -m pip install --upgrade pip' command.


In [24]:
state_dist=[]
for s in range(nS):
    c=w_obj.weight_obj(s).detach().cpu().numpy()[0]
    state_dist.append(c)
c=np.array(state_dist)*behaviour_policy_state_distribution

In [25]:
c/np.sum(c)

array([0.49553907, 0.15608018, 0.14004363, 0.20833712])

In [ ]:
pd.DataFrame(data[T_update-1]).to_excel("Data_Used_Thompson_Sampling_variant_2.xlsx");
pd.DataFrame(estimated_value).to_excel("Estimated_Value_functions_Thompson_Sampling_variant_2.xlsx");

In [ ]:
len(data[T_update-1])

In [347]:
a = np.ones(10)
a[1] = 3
a[4] = 5
a
b = np.reshape(a,(5,2))

In [348]:
a

array([1., 3., 1., 1., 5., 1., 1., 1., 1., 1.])

In [349]:
b

array([[1., 3.],
       [1., 1.],
       [5., 1.],
       [1., 1.],
       [1., 1.]])